# 最初のグラフの生成とセッション内での実行

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

In [9]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
sess.close()
print(result)

42


In [10]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
print(result)

42


In [12]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()
print(result)

42


In [13]:
sess = tf.InteractiveSession()

init.run()
result = f.eval()
sess.close()
print(result)

42


# グラフの管理

In [14]:
x1 = tf.Variable(1)

x1.graph is tf.get_default_graph()

True

In [26]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
    print(x2.graph is tf.get_default_graph())

print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())


True
True
False


# ノードの値のライフサイクル

In [27]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [28]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


# TensorFlowによる線形回帰

In [31]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
print(housing.data)
print(housing_data_plus_bias)

[[   8.3252       41.            6.98412698 ...    2.55555556
    37.88       -122.23      ]
 [   8.3014       21.            6.23813708 ...    2.10984183
    37.86       -122.22      ]
 [   7.2574       52.            8.28813559 ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.7          17.            5.20554273 ...    2.3256351
    39.43       -121.22      ]
 [   1.8672       18.            5.32951289 ...    2.12320917
    39.43       -121.32      ]
 [   2.3886       16.            5.25471698 ...    2.61698113
    39.37       -121.24      ]]
[[   1.            8.3252       41.         ...    2.55555556
    37.88       -122.23      ]
 [   1.            8.3014       21.         ...    2.10984183
    37.86       -122.22      ]
 [   1.            7.2574       52.         ...    2.80225989
    37.85       -122.24      ]
 ...
 [   1.            1.7          17.         ...    2.3256351
    39.43       -121.22      ]
 [   1.            1.8672       18.         ...    2.12320917

In [32]:
X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
print(housing.target)
print(housing.target.reshape(-1, 1))

[4.526 3.585 3.521 ... 0.923 0.847 0.894]
[[4.526]
 [3.585]
 [3.521]
 ...
 [0.923]
 [0.847]
 [0.894]]


In [34]:
XT = tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)

with tf.Session() as sess:
    theta_value = theta.eval()

theta_value

array([[-3.7465141e+01],
       [ 4.3573415e-01],
       [ 9.3382923e-03],
       [-1.0662201e-01],
       [ 6.4410698e-01],
       [-4.2513184e-06],
       [-3.7732250e-03],
       [-4.2664889e-01],
       [-4.4051403e-01]], dtype=float32)

# 勾配降下法の実装

## マニュアルの勾配計算

In [41]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = 2 / m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 11.325348
Epoch 100 MSE = 4.9488482
Epoch 200 MSE = 4.888695
Epoch 300 MSE = 4.8650312
Epoch 400 MSE = 4.848227
Epoch 500 MSE = 4.836048
Epoch 600 MSE = 4.827201
Epoch 700 MSE = 4.8207726
Epoch 800 MSE = 4.816094
Epoch 900 MSE = 4.812681


## 自動微分を使った方法

In [42]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
gradients = tf.gradients(mse, [theta])[0]
training_op = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 7.3161187
Epoch 100 MSE = 5.007195
Epoch 200 MSE = 4.9328346
Epoch 300 MSE = 4.896245
Epoch 400 MSE = 4.8703127
Epoch 500 MSE = 4.8516293
Epoch 600 MSE = 4.8381634
Epoch 700 MSE = 4.8284397
Epoch 800 MSE = 4.821425
Epoch 900 MSE = 4.8163614


## オプティマイザを使うと

In [44]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 10.164854
Epoch 100 MSE = 5.1071
Epoch 200 MSE = 5.0046377
Epoch 300 MSE = 4.9482555
Epoch 400 MSE = 4.907886
Epoch 500 MSE = 4.878766
Epoch 600 MSE = 4.857755
Epoch 700 MSE = 4.8425956
Epoch 800 MSE = 4.831656
Epoch 900 MSE = 4.823761


In [46]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 7.6721983
Epoch 100 MSE = 4.8059125
Epoch 200 MSE = 4.8033953
Epoch 300 MSE = 4.803269
Epoch 400 MSE = 4.803256
Epoch 500 MSE = 4.8032565
Epoch 600 MSE = 4.8032537
Epoch 700 MSE = 4.803252
Epoch 800 MSE = 4.803252
Epoch 900 MSE = 4.803251


# 訓練アルゴリズムへのデータの供給

In [51]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
    
print(B_val_1)
print(B_val_2)

[[ 6.  9. 14.]]
[[21. 30. 41.]
 [54. 69. 86.]]


In [57]:
from sklearn.model_selection import train_test_split

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

#print("n=", n)

def fetch_batch(epoch, batch_index, batch_size):
    X_train, X_test, y_train, y_test = train_test_split(scaled_housing_data_plus_bias, housing.target)
#    print(X_train.shape)
#    print(y_train.shape)
    return X_train, y_train.reshape(-1, 1)

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        print("Epoch=", epoch)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    print(best_theta)

Epoch= 0
Epoch= 1
Epoch= 2
Epoch= 3
Epoch= 4
Epoch= 5
Epoch= 6
Epoch= 7
Epoch= 8
Epoch= 9
Epoch= 10
Epoch= 11
Epoch= 12
Epoch= 13
Epoch= 14
Epoch= 15
Epoch= 16
Epoch= 17
Epoch= 18
Epoch= 19
Epoch= 20
Epoch= 21
Epoch= 22
Epoch= 23
Epoch= 24
Epoch= 25
Epoch= 26
Epoch= 27
Epoch= 28
Epoch= 29
Epoch= 30
Epoch= 31
Epoch= 32
Epoch= 33
Epoch= 34
Epoch= 35
Epoch= 36
Epoch= 37
Epoch= 38
Epoch= 39
Epoch= 40
Epoch= 41
Epoch= 42
Epoch= 43
Epoch= 44
Epoch= 45
Epoch= 46
Epoch= 47
Epoch= 48
Epoch= 49
Epoch= 50
Epoch= 51
Epoch= 52
Epoch= 53
Epoch= 54
Epoch= 55
Epoch= 56
Epoch= 57
Epoch= 58
Epoch= 59
Epoch= 60
Epoch= 61
Epoch= 62
Epoch= 63
Epoch= 64
Epoch= 65
Epoch= 66
Epoch= 67
Epoch= 68
Epoch= 69
Epoch= 70
Epoch= 71
Epoch= 72
Epoch= 73
Epoch= 74
Epoch= 75
Epoch= 76
Epoch= 77
Epoch= 78
Epoch= 79
Epoch= 80
Epoch= 81
Epoch= 82
Epoch= 83
Epoch= 84
Epoch= 85
Epoch= 86
Epoch= 87
Epoch= 88
Epoch= 89
Epoch= 90
Epoch= 91
Epoch= 92
Epoch= 93
Epoch= 94
Epoch= 95
Epoch= 96
Epoch= 97
Epoch= 98
Epoch= 99
Epoch= 100

Epoch= 755
Epoch= 756
Epoch= 757
Epoch= 758
Epoch= 759
Epoch= 760
Epoch= 761
Epoch= 762
Epoch= 763
Epoch= 764
Epoch= 765
Epoch= 766
Epoch= 767
Epoch= 768
Epoch= 769
Epoch= 770
Epoch= 771
Epoch= 772
Epoch= 773
Epoch= 774
Epoch= 775
Epoch= 776
Epoch= 777
Epoch= 778
Epoch= 779
Epoch= 780
Epoch= 781
Epoch= 782
Epoch= 783
Epoch= 784
Epoch= 785
Epoch= 786
Epoch= 787
Epoch= 788
Epoch= 789
Epoch= 790
Epoch= 791
Epoch= 792
Epoch= 793
Epoch= 794
Epoch= 795
Epoch= 796
Epoch= 797
Epoch= 798
Epoch= 799
Epoch= 800
Epoch= 801
Epoch= 802
Epoch= 803
Epoch= 804
Epoch= 805
Epoch= 806
Epoch= 807
Epoch= 808
Epoch= 809
Epoch= 810
Epoch= 811
Epoch= 812
Epoch= 813
Epoch= 814
Epoch= 815
Epoch= 816
Epoch= 817
Epoch= 818
Epoch= 819
Epoch= 820
Epoch= 821
Epoch= 822
Epoch= 823
Epoch= 824
Epoch= 825
Epoch= 826
Epoch= 827
Epoch= 828
Epoch= 829
Epoch= 830
Epoch= 831
Epoch= 832
Epoch= 833
Epoch= 834
Epoch= 835
Epoch= 836
Epoch= 837
Epoch= 838
Epoch= 839
Epoch= 840
Epoch= 841
Epoch= 842
Epoch= 843
Epoch= 844
Epoch= 845

# モデルの保存と復元

In [60]:
from sklearn.model_selection import train_test_split

n_epochs = 10
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()
saver_theta = tf.train.Saver({"weights": theta})

#print("n=", n)

def fetch_batch(epoch, batch_index, batch_size):
    X_train, X_test, y_train, y_test = train_test_split(scaled_housing_data_plus_bias, housing.target)
#    print(X_train.shape)
#    print(y_train.shape)
    return X_train, y_train.reshape(-1, 1)

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            save_path = saver.save(sess, "./trained_model/my_model.ckpt")
            save_theta_path = saver_theta.save(sess, ".trained_model/my_model_theta.ckpt")
            
        print("Epoch=", epoch)
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./trained_model/my_model_finally.ckpt")
    save_theta_path = saver_theta.save(sess, ".trained_model/my_model_theta_finally.ckpt")
    print("best_theta:", best_theta)
    print("save_path:", save_path)
    
    with tf.Session() as sess:
        saver.restore(sess, "./trained_model/my_model_finally.ckpt")
        


Epoch= 0
Epoch= 1
Epoch= 2
Epoch= 3
Epoch= 4
Epoch= 5
Epoch= 6
Epoch= 7
Epoch= 8
Epoch= 9
best_theta: [[ 0.18321323]
 [ 0.8473657 ]
 [ 0.12728973]
 [-0.29153892]
 [ 0.32288796]
 [-0.0020721 ]
 [-0.03923169]
 [-0.8231216 ]
 [-0.7950508 ]]
save_path: ./trained_model/my_model_finally.ckpt
INFO:tensorflow:Restoring parameters from ./trained_model/my_model_finally.ckpt


# TensorBoardを使ったグラフと訓練曲線の可視化

In [ ]:
from datetime import datetime
import tensorflow as tf
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}".format(root_logdir, now)

scaler = StandardScaler()

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data]
scaled_housing_data_plus_bias = scaler.fit_transform(housing_data_plus_bias)

n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

saver = tf.train.Saver()
saver_theta = tf.train.Saver({"weights": theta})

mse_summary = tf.summary.scalar("MSE", mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

def fetch_batch(epoch, batch_index, batch_size):
    X_train, X_test, y_train, y_test = train_test_split(scaled_housing_data_plus_bias, housing.target)
    
    return X_train, y_train.reshape(-1, 1)

with tf.Session() as sess:
    init.run()
    
    for epoch in range(n_epochs):
        if epoch % 10 == 0:
            save_path = saver.save(sess, "./trained_model/my_model.ckpt")
            save_theta_path = saver_theta.save(sess, ".trained_model/my_model_theta.ckpt")
            
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "./trained_model/my_model_finally.ckpt")
    save_theta_path = saver_theta.save(sess, ".trained_model/my_model_theta_finally.ckpt")
    
    with tf.Session() as sess:
        saver.restore(sess, "./trained_model/my_model_finally.ckpt")
        
file_writer.close()